<a href="https://colab.research.google.com/github/kazuki-de/git_reserch_code/blob/master/VAT%EF%BC%BFKL_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-beta1
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Activation , Softmax, MaxPooling2D,GlobalAveragePooling2D, Dropout, BatchNormalization


from tensorflow.keras import Model
import datetime
import tensorflow_probability as tfp

In [2]:
from google.colab import drive
drive.mount("/content/drive")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [4]:
cd  drive/My Drive/logs

/content/drive/My Drive/logs


In [0]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="/content/drive/My Drive/logs/gradient_tape", histogram_freq=1)

In [0]:
cifar10 = tf.keras.datasets.cifar10
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train, x_test = x_train / 255.0, x_test / 255.0



# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


In [0]:
#ラベルのカテゴリカル化
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

<BatchDataset shapes: ((None, 32, 32, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [0]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(64, (3,3), padding="same", activation='relu')
        self.conv1_2 = Conv2D(64, (3,3), padding="same", activation='relu')
        self.maxpooling1 = MaxPooling2D()
        
        
        self.conv2 = Conv2D(128, (3,3), padding="same", activation='relu')
        self.conv2_2 = Conv2D(128, (3,3), padding="same", activation='relu')
        self.conv2_3 = Conv2D(128, (3,3), padding="same", activation='relu')
        self.maxpooling2 = MaxPooling2D()
        
        self.conv3 = Conv2D(256,(3,3), padding="same",activation="relu")
        self.conv3_2 = Conv2D(256,(3,3), padding="same",activation="relu")
        self.conv3_3 = Conv2D(256,(3,3), padding="same",activation="relu")
        self.conv3_4 = Conv2D(256,(3,3), padding="same",activation="relu")
        self.conv3_5 = Conv2D(256,(3,3), padding="same",activation="relu")
        self.maxpooling3 = MaxPooling2D()
        
        self.conv4 = Conv2D(512,(3,3), padding="same",activation="relu")
        self.conv4_2 = Conv2D(512,(3,3), padding="same",activation="relu")
        
        
        self.flatten = Flatten()
        self.d1_1 = Dense(1024, activation="relu")
        self.d1_2 = Dense(1024, activation='relu')
        
        self.d2 = Dense(10)
        self.dropout1 = Dropout(0.25)
        self.dropout2 = Dropout(0.5)
        
        self.GAPooling = GlobalAveragePooling2D()
        self.BatchN = BatchNormalization()
        self.BatchN_2 = BatchNormalization()
        self.BatchN_3 = BatchNormalization()
        self.BatchN_4 = BatchNormalization()

    def call(self, x):
        x = self.conv1(x)
        x = self.BatchN(x)
        x = self.conv1_2(x)
        x = self.maxpooling1(x)
        x = self.dropout1(x)
        
        x = self.conv2(x)
        x = self.conv2_2(x)
        x = self.BatchN_2(x)
        x = self.conv2_3(x)
        x = self.maxpooling2(x)
        x = self.dropout1(x)
        
        x = self.conv3(x)
        x = self.conv3_2(x)
        x = self.BatchN_3(x)
        x = self.conv3_3(x)
        x = self.conv3_4(x)
        x = self.conv3_5(x)
        x = self.BatchN_4(x)
        
        x = self.conv4(x)
        x = self.conv4_2(x)
        x = self.GAPooling(x)
        x = self.d1_1(x)
        x = self.dropout2(x)
        x = self.d1_2(x)
        x = self.dropout2(x)
        
        return self.d2(x)

model = MyModel()

In [0]:
class MyModel_2(Model):
    def __init__(self):
        super(MyModel_2, self).__init__()
        self.conv1 = Conv2D(64, (3,3), padding="same", activation='relu')
        self.conv1_2 = Conv2D(64, (3,3), padding="same", activation='relu')
        self.maxpooling1 = MaxPooling2D()
        
        
        self.conv2 = Conv2D(128, (3,3), padding="same", activation='relu')
        self.conv2_2 = Conv2D(128, (3,3), padding="same", activation='relu')
        self.conv2_3 = Conv2D(128, (3,3), padding="same", activation='relu')
        self.maxpooling2 = MaxPooling2D()
        
        self.conv3 = Conv2D(256,(3,3), padding="same",activation="relu")
        self.conv3_2 = Conv2D(256,(3,3), padding="same",activation="relu")
        self.conv3_3 = Conv2D(256,(3,3), padding="same",activation="relu")
        self.conv3_4 = Conv2D(256,(3,3), padding="same",activation="relu")
        self.conv3_5 = Conv2D(256,(3,3), padding="same",activation="relu")
        self.maxpooling3 = MaxPooling2D()
        
        self.conv4 = Conv2D(512,(3,3), padding="same",activation="relu")
        self.conv4_2 = Conv2D(512,(3,3), padding="same",activation="relu")
        
        
        self.flatten = Flatten()
        self.d1_1 = Dense(1024, activation="relu")
        self.d1_2 = Dense(1024, activation='relu')
        
        self.d2 = Dense(10)
        self.dropout1 = Dropout(0.25)
        self.dropout2 = Dropout(0.5)
        
        self.GAPooling = GlobalAveragePooling2D()
        self.BatchN = BatchNormalization()
        self.BatchN_2 = BatchNormalization()
        self.BatchN_3 = BatchNormalization()
        self.BatchN_4 = BatchNormalization()

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1_2(x)
        x = self.dropout2(x)
        
        return self.d2(x)

model = MyModel_2()

In [0]:
#損失関数
loss_object = tf.keras.losses.CategoricalCrossentropy()
#最適化手法
optimizer = tf.keras.optimizers.Adam()

#評価関数
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [0]:
def VAT_cifar10(input_tensor, network, xi=1e-6, epsilon=2.0, weight=1.0, num_approximation=1, clip_value_min=1e-30, dtype=tf.float64):
    #tf.reduce_sum 配列内の対応した数を足しあわせる
    clipped = lambda x: tf.maximum(x, clip_value_min)

    axis_without_batch_size = tuple(range(1,len(input_tensor.get_shape())))
    if len(axis_without_batch_size) == 1: axis_without_batch_size = axis_without_batch_size[0]
    normalized = lambda x: x / clipped(tf.norm(x, axis=None))
    plain_softmax = network(input_tensor)
    perturbation = xi * normalized(tf.random.normal(shape=tf.shape(input_tensor), dtype=dtype))
    for i in range(num_approximation):
        softmax_accommodating_perturbation = network(input_tensor+perturbation)
        #8/8----------------------------------------------------------------------------------------------------
        
        cross_entropy_accommodating_perturbation = -tf.reduce_sum(plain_softmax * tf.math.log(clipped(softmax_accommodating_perturbation))) * weight
        adversarial_direction = tf.gradients(cross_entropy_accommodating_perturbation, [perturbation])[0]
        vat_perturbation = normalized(adversarial_direction)
        perturbation = xi * vat_perturbation
        
        
    current_softmax = Activation('softmax')(network(input_tensor)) 
    current_softmax = tf.stop_gradient(current_softmax)
    vat_perturbation = tf.stop_gradient(epsilon * vat_perturbation)
    vat_softmax = Activation('softmax')(network(input_tensor + vat_perturbation))
    print(current_softmax)
    bbb=tf.cast(tf.reduce_sum(weight)+1e-30, tf.float64)
    
    vat_cross_entropy = tf.reduce_sum(-tf.reduce_sum(current_softmax * tf.math.log(clipped(vat_softmax))) * weight) / bbb
    return vat_cross_entropy

In [0]:
def VAT_mnist(input_tensor, network, xi=1e-6, epsilon=2.0, weight=1.0, num_approximation=1, clip_value_min=1e-30, dtype=tf.float64):
    
    #配列内の数値の最大値が最大第二変数になるように置き換える
    clipped = lambda x: tf.maximum(x, clip_value_min)
    #axis_without_batch_size は (1,2,3)となる
    axis_without_batch_size = tuple(range(1,len(input_tensor.get_shape())))
    
    if len(axis_without_batch_size) == 1:
        axis_without_batch_size = axis_without_batch_size[0]
        
    #normalized = lambda x: x / clipped(tf.norm(x, axis=None))
    normalized = lambda x: x / clipped(tf.norm(x, axis=None,keepdims=True))
    
    plain_softmax = network(input_tensor)
    perturbation = xi * normalized(tf.random.normal(shape=tf.shape(input_tensor), dtype=dtype))
    for i in range(num_approximation):
        softmax_accommodating_perturbation = network(input_tensor+perturbation)
        #8/8----------------------------------------------------------------------------------------------------
         #tf.reduce_sum それぞれの配列内に対応した数を足しあわせる
        cross_entropy_accommodating_perturbation = -tf.reduce_sum(plain_softmax * tf.math.log(clipped(softmax_accommodating_perturbation))) * weight
        print(cross_entropy_accommodating_perturbation)
        adversarial_direction = tf.gradients(cross_entropy_accommodating_perturbation, [perturbation])[0]
        vat_perturbation = normalized(adversarial_direction)
        perturbation = xi * vat_perturbation
        
        
    current_softmax = Activation('softmax')(network(input_tensor)) 
    current_softmax = tf.stop_gradient(current_softmax)
    vat_perturbation = tf.stop_gradient(epsilon * perturbation)
    vat_softmax = Activation('softmax')(network(input_tensor + perturbation))
    #print(current_softmax)
    bbb=tf.cast(tf.reduce_sum(weight)+1e-30, tf.float64)
    
    vat_cross_entropy = tf.reduce_sum(-tf.reduce_sum(current_softmax * tf.math.log(clipped(vat_softmax))) * weight) / bbb
    return vat_cross_entropy

In [0]:
def kl_divergence(q_logit, p_logit):
    qlogq = tf.reduce_mean(tf.reduce_sum(q_logit * tf.math.log(q_logit), 1))
    qlogp = tf.reduce_mean(tf.reduce_sum(q_logit * tf.math.log(p_logit), 1))
    return qlogq - qlogp

In [0]:
def VAT_KL(input_tensor, network, xi=10, epsilon=1.0, weight=1.0, num_approximation=1, clip_value_min=1e-30, dtype=tf.float32):
    
    #配列内の数値の最大値が第二変数になるように置き換える
    clipped = lambda x: tf.maximum(x, clip_value_min)
    #axis_without_batch_size は (1,2,3)となる
    #axis_without_batch_size = tuple(range(1,len(input_tensor.get_shape())))
    
    #if len(axis_without_batch_size) == 1:
    #    axis_without_batch_size = axis_without_batch_size[0]
        
    #normalized = lambda x: x / clipped(tf.norm(x, axis=None))
    normalized = lambda x: x / clipped(tf.norm(x, axis=None,keepdims=True))
    
    plain_softmax = network(input_tensor)
    
    noplain_softmax = Activation("softmax")(plain_softmax)
    
    #適当な単位ベクトルにxi=10を掛けた数:pertubationの作成
    perturbation = xi * normalized(tf.random.normal(shape=tf.shape(input_tensor), dtype=dtype))
    
    
    for i in range(num_approximation):
        
        softmax_accommodating_perturbation = network(input_tensor+perturbation)
        softmax_accommodating_perturbation_af = Activation("softmax")(softmax_accommodating_perturbation)
        # ノイズを足した配列とノーマルな配列がCNNで出力された結果のKL距離を求める
        dist = kl_divergence(noplain_softmax,softmax_accommodating_perturbation_af)
        #cross_entropy_accommodating_perturbation = -tf.reduce_sum(plain_softmax * tf.math.log(clipped(softmax_accommodating_perturbation))) * weight
        adversarial_direction = tf.gradients(dist, [perturbation])[0]
        pertubation = tf.stop_gradient(adversarial_direction)
    print("ok")    
    pertubation = epsilon * normalized(pertubation)
    corrent_softmax = Activation('softmax')(network(input_tensor)) 
    vat_softmax = Activation('softmax')(network(input_tensor + perturbation))
    loss = kl_divergence(corrent_softmax,vat_softmax)
    return loss
        

In [0]:
@tf.function
def train_step(image, label):
    with tf.GradientTape() as tape:
        predictions = model(image)
        softmax = Activation('softmax')(predictions)
        loss = loss_object(label, softmax)
        print("ok")
        VAT_loss = VAT_KL(image,model)
        loss_add = loss + VAT_loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(label, softmax)


@tf.function
def test_step(image, label):
    predictions = model(image)
    softmax = Activation('softmax')(predictions)
    t_loss = loss_object(label, softmax)

    test_loss(t_loss)
    test_accuracy(label, softmax)



In [108]:
import time
EPOCHS = 200
model = MyModel()

for epoch in range(EPOCHS):
    start = time.time()
    for image, label in train_ds:
        train_step(image, label)
        
    #with train_summary_writer.as_default():
        #tf.summary.scalar('loss', train_loss.result(), step=epoch)
        #tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
    
    
    for test_image, test_label in test_ds:
        test_step(test_image, test_label)
    #with test_summary_writer.as_default():
        #tf.summary.scalar('loss', test_loss.result(), step=epoch)
        #tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
    elapsed_time = time.time()-start
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100)
                         )
    print(elapsed_time)
    
train_loss.reset_states()
test_loss.reset_states()
train_accuracy.reset_states()
test_accuracy.reset_states()

Epoch 1, Loss: 0.06258056312799454, Accuracy: 98.0875015258789, Test Loss: 0.05464131012558937, Test Accuracy: 98.29000091552734
17.336491346359253
Epoch 2, Loss: 0.04805632680654526, Accuracy: 98.51944732666016, Test Loss: 0.0547214038670063, Test Accuracy: 98.3566665649414
17.302776336669922
Epoch 3, Loss: 0.03951670229434967, Accuracy: 98.77708435058594, Test Loss: 0.05920248106122017, Test Accuracy: 98.38999938964844
17.34789752960205


KeyboardInterrupt: ignored

In [0]:
x = tf.norm(x_train[1], axis=None,keepdims=True)
x

In [0]:
clipped = lambda x: tf.maximum(x, 1e-30)
#axis_without_batch_size は (1,2,3)となる
axis_without_batch_size = (1,2,3)

if len(axis_without_batch_size) == 1:
    axis_without_batch_size = axis_without_batch_size[0]

#normalized = lambda x: x / clipped(tf.norm(x, axis=None))
normalized = lambda x: x / clipped(tf.norm(x, axis=None))

#plain_softmax = network(input_tensor)
perturbation = 1* normalized(tf.random.normal(shape=tf.shape(x_train[1:12]), dtype=tf.float32))

In [0]:
perturbation[1].shape

TensorShape([28, 28, 1])

In [0]:
x

(1, 2, 3)

In [0]:
#tensorboard --logdir content/drive/My Drive/logs/gradient_tape

tensorboard --logdir= /content/drive/My Drive/logs/gradient_tape

SyntaxError: ignored

In [0]:
predict = model(x_test)
predict1 = tf.argmax(predict,axis=1).numpy()

In [0]:
import numpy as np
x = np.array( [] )
count = 0
count2 = 0

for i in range(len(x_train)+1):
    if count % 1000 ==0:
        g = x_train[count-1000:count]
        predict = model(g)
        predict1 = tf.argmax(predict,axis=1).numpy()
        x = np.append( x, predict1 )
    count+=1
    if i ==59999:
        print("ok")

ok


In [0]:
a = 1
a.type()

AttributeError: ignored